In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [2]:
from PyPDF2 import PdfReader

reader = PdfReader('pdf\Revue-Médias - DR Nord-Pas-de-Calais du 19122024.pdf')
page = reader.pages[5]
extracted_text = page.extract_text()
print(extracted_text =='DR Nord-Pas-de-Calais')

False


In [3]:
from PyPDF2 import PdfReader

def extract_articles_from_pdf(pdf_path, dossier=False):
    # Initialisation du lecteur de PDF
    reader = PdfReader(pdf_path)
    all_text = ''
    
    # Liste pour stocker les pages qui seront incluses dans les articles
    pages_text = []
    
    # Si dossier=True, ignorer les pages avant la première page contenant 'DR Nord-Pas-de-Calais' uniquement
    if dossier:
        found_first_article = False
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page_text = page.extract_text()
            
            # Si la page contient seulement 'DR Nord-Pas-de-Calais', l'ignorer
            if page_text.strip() == 'DR Nord-Pas-de-Calais':
                found_first_article = True
                continue  # Passer à la page suivante sans ajouter cette page
            
            elif found_first_article:  # Ajouter les pages pertinentes après la première page d'article
                pages_text.append(page_text)
    else:
        # Si dossier=False, on ajoute simplement toutes les pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page_text = page.extract_text()
            pages_text.append(page_text)
    
    # Joindre toutes les pages extraites en un seul texte
    all_text = "\n".join(pages_text)
    
    # Expression régulière pour identifier les articles en fonction de la balise "Parution"
    # Nous recherchons "Parution" pour délimiter les fins d'article
    articles = []
    current_article = []
    
    # Séparer le texte sur chaque occurrence de la balise "Parution"
    for page_num, page_text in enumerate(pages_text):
        # Si la page contient la balise "Parution"
        if "Parution" in page_text:
            # Ajouter la page courante contenant la balise "Parution"
            current_article.append(page_text)
            
            # Ajouter l'article complet à la liste des articles
            articles.append("\n".join(current_article))
            
            # Réinitialiser pour commencer un nouvel article
            current_article = []
        else:
            # Ajouter la page à l'article en cours
            current_article.append(page_text)
    
    # Si un article est encore en cours après la dernière page, l'ajouter aussi
    if current_article:
        articles.append("\n".join(current_article))
    
    # Nettoyer les articles en enlevant les espaces superflus
    articles = [article.strip() for article in articles if article.strip()]
    
    return articles

# Exemple d'utilisation
pdf_path = 'pdf\Revue-Médias - DR Nord-Pas-de-Calais du 12122024.pdf'

# Si dossier=True, les pages avant la page contenant 'DR Nord-Pas-de-Calais' sont exclues
articles = extract_articles_from_pdf(pdf_path, dossier=True)

# Affichage du nombre d'articles extraits
print(f'Nombre d\'articles extraits : {len(articles)}')


Nombre d'articles extraits : 42


In [4]:
# Affichage du premier article (à titre d'exemple)
print(articles[0])

REGARDER (00:00:44) 
Émission du 7 décembre 2024 de 17:00 à 17:30
[Reportage] Tempête Darragh : 1100 coupures d'électricité
enregistrées dans le nord à 16h
    
Mot(s) clé(s) : 
Reportage, Enedis DR Nord Pas de Calais, Tempête Darragh, 1100 coupures d'électricité 
17:01:58  -  [Reportage]  La  tempête  Darragh  a  soufflé  fort  dans  le  Nord-Pas-de-Calais  :  1100
coupures d'électricité étaient enregistrées par Enedis à 16h. Des arbres sont tombés sur les lignes à
cause du vent. Une cinquantaine d'agents sont mobilisés. 
17:02:37  - Fin. 
↑ 9
SUR LE VIF
Quel hélico ?
 jeudi 12 au jeudi 19 décembre 2024
Édition(s) : Edition Principale
Page 2
34 mots - < 1 min
  
   
Dans les jours à venir, vous
pourrez  apercevoir  un
hélicoptère survoler le secteur.C’est Enedis qui surveillera les
installations électriques, possi‐
blement jusque fin janvier.  ■
T ous droits réservés L'Echo de la Lys 2024
5967e9a08dc0520d709930f6a305318a080AadB44Qa9X9eb28b1
613
 Parution : Hebdomadaire
Diffusion : 468

### Extraction de la date

In [5]:
import re

# Fonction pour extraire la date en pleine lettre dans le texte
def extract_date_from_text(text):
    # Expression régulière pour capturer la date en pleine lettre (jour, mois, année)
    date_pattern = r'\b(?:lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche)\s+(\d{1,2})\s+([a-zA-Zéàôù]+)\s+(\d{4})\b'
    
    # Recherche de la date avec l'expression régulière
    match = re.search(date_pattern, text)
    if match:
        # Extraire le jour, le mois et l'année
        day, month, year = match.groups()
        
        # Convertir le mois en texte à son format numérique
        months = {
            'janvier': '01', 'février': '02', 'mars': '03', 'avril': '04',
            'mai': '05', 'juin': '06', 'juillet': '07', 'août': '08',
            'septembre': '09', 'octobre': '10', 'novembre': '11', 'décembre': '12'
        }
        
        # Récupérer le mois au format numérique
        month_num = months.get(month.lower())
        
        if month_num:
            # Construire la date au format mois/jour/année
            date_str = f"{month_num}/{int(day):02d}/{year}"
            
            # Retourner la date formatée
            return date_str
    
    # Si aucune date n'est trouvée
    return None

# Test de la fonction
article = articles[-1]

# Extraction de la date
date_extracted = extract_date_from_text(article)
print(f"Date extraite : {date_extracted}")


Date extraite : 12/05/2024


### Extraction du nom du journal

In [6]:
from media_ressources import media_dict

def check_media_in_text(text):
    # Normalisation du texte pour une comparaison insensible à la casse
    text_lower = text.lower()

    # Liste des variantes de "Tous droits réservés" à détecter
    reserved_rights_variations = [
        "tous droits réservés",
        "t ous droits réservés",  # Possible typo with extra space
        "tous droits reserve",    # Sans accent
        "tous droits réservée",    # Petite erreur grammaticale possible
        "tous droit réservé",      # Singulier
        "tous droit réservés",     # Mélange singulier/pluriel
        "© tous droits réservés",  # Avec le symbole copyright
        "(c) tous droits réservés",
        "tous droits réservés.",   # Avec un point final
        "tous droits réservés :",
    ]

    # Trouver la première occurrence de "Tous droits réservés" parmi ses variantes
    reserved_rights_index = -1
    for variant in reserved_rights_variations:
        index = text_lower.find(variant)
        if index != -1 and (reserved_rights_index == -1 or index < reserved_rights_index):
            reserved_rights_index = index

    # Liste des médias trouvés
    found_media = []
    
    # Vérification de chaque média dans le dictionnaire
    for media, variations in media_dict.items():
        for variation in variations:
            # Vérification d'une correspondance exacte de chaque variation dans le texte
            variation_lower = variation.lower()
            index = text_lower.find(variation_lower)

            if index != -1:  # Si la variation est trouvée dans le texte
                if reserved_rights_index != -1 and index > reserved_rights_index:
                    # Si "Tous droits réservés" précède ce média, on retourne uniquement celui-ci
                    return [media]
                found_media.append(media)
                break  # Une fois le média trouvé, on passe au suivant

    return found_media  # Retourne la liste complète si "Tous droits réservés" n'est pas trouvé



for article in articles:
    resultats = check_media_in_text(article)
    print(resultats)

["L'Écho de la Lys"]
["L'Écho de la Lys"]
['La Voix du Nord']
['La Voix du Nord']
['Nord Éclair']
['Nord Éclair']
['La Voix du Nord']
['Nord Éclair']
["L'Observateur"]
["L'Abeille de la Ternoise"]
['Nord Éclair']
['La Voix du Nord']
['La Voix du Nord']
['La Semaine dans le Boulonnais']
['Le Journal des Flandres']
['La Voix du Nord']
['La Voix du Nord']
['Nord Éclair']
['Nord Littoral']
['Nord Littoral']
['Nord Littoral']
['Nord Littoral']
['La Voix du Nord']
['La Voix du Nord']
['Nord Littoral']
['La Voix du Nord']
['La Voix du Nord']
['Nord Littoral']
['La Voix du Nord']
['Nord Éclair']
['Nord Littoral']
['La Voix du Nord']
['Nord Littoral']
['La Voix du Nord']
['Nord Littoral']
['La Voix du Nord']
['Nord Éclair']
['La Voix du Nord']
["L'Observateur"]
["L'Observateur"]
['La Voix du Nord']
['La Voix du Nord']


In [7]:
article = articles[9]
check_media_in_text(article)
print(article)

MAREST // SUR UN CHANTIER RUE DE CAMBLAIN
Un ouvrier électrocuté 
Mercredi 4 décembre vers 15 heures 45, les sapeurspompiers ont porté secours à
deux ouvriers victimes d'une électrocution sur un chantier rue de Camblain.
 N° 2450
jeudi 12 au jeudi 19 décembre 2024
Page 4
180 mots - 1 min
  
   
L'ouvrier forait un trou quand il
aurait touché une ligne haute ten‐
sion.
 
Aux  manettes  d'un  engin  de
chantier de forage, un homme
aurait  touché  un  câble  haute
tension souterrain. À l'arrivée
des  sapeurs-  pompiers  dePernes, Auchel et l'infirmier sa‐
peur-pompier  de  Lillers,  re‐
joints par une équipe médicale
du Smur de Béthune, l'ouvrier
(35 ans) était en arrêt cardio‐
respiratoire.  Les  secours  sont
parvenus à le réanimer avant
son  transport  médicalisé  au
centre hospitalier d'Arras. Son
pronostic vital était engagé.
Son collègue (32 ans) a été di‐
rigé vers la polyclinique de laClarence à Divion légèrement
blessé.
Une enquête de gendarmerie a
été  ouverte  pour  déterminer
